# Therapy System 

## Summary 

a discrete-event simulation (using SimPy) of a therapy system:

- Patients arrive over time (3 years)
- Only a fraction actually start therapy (PATIENT_START_THERAPY)
- Therapy capacity is limited by a fixed number of therapy slots (resources)
- If all slots are busy, patients wait in a queue
- Each patient who gets a slot occupies it for ~30 weeks
- Demand increases over time (growth)
- The code measures the waiting time of patients who start therapy

### Basis Simulation 
scenario_adjustment = 0,95

In [ ]:
import simpy
import numpy as np

# -----------------------------
# Configuration 
# -----------------------------
WEEKS_PER_YEAR = 52
SIMULATION_DURATION = int(WEEKS_PER_YEAR * 3) #simulation runs for 3 years, measured in weeks
SEED = 42

#------------------------------
## Input
TOTAL_PATIENTS_YEAR = 95201   # people “show up” per year
YES_PATIENTS_YEAR = 84557     # only YES_PATIENTS_YEAR actually start therapy
PATIENT_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR #random arrival starts therapy

## Therapy duration (in weeks)
SESSIONS_REQUIRED = 24        # Therapy needs 24 sessions
WORKING_WEEKS = 42            # 42 working weeks/year (vacation/sick leave removed)
TIME_STRETCH = WEEKS_PER_YEAR / WORKING_WEEKS # “session week” is stretched by 52/42
REAL_DURATION_WEEKS = SESSIONS_REQUIRED * TIME_STRETCH 
# a therapy course occupies a slot for about 29.7 week

## Capacity calculation
# theoretical maximum number of parallel therapy slots based on staff groups
SLOTS_PART_TIME = 5490 * 9
SLOTS_FULL_TIME = 3510 * 15
THEORETICAL_CAPACITY = SLOTS_PART_TIME + SLOTS_FULL_TIME # ~102.060

### Slots needed for 100% coverage
    # Each year YES_PATIENTS_YEAR therapy starts
    # Patient start occupies a slot for REAL_DURATION_WEEKS
    # Divide by 52 to convert to “average concurrent slots needed”
REQUIRED_SLOTS_SATURATION = (YES_PATIENTS_YEAR * REAL_DURATION_WEEKS) / WEEKS_PER_YEAR

### Break- even factor
    # theoretical capacity you’d need so that capacity equals demand (100% coverage)
    # converts that to an absolute number of slots 
BREAK_EVEN_FACTOR = REQUIRED_SLOTS_SATURATION / THEORETICAL_CAPACITY

### Scenario adjustment
    # 1.00 = Perfect (no waiting lines)
    # 0.95 = 5% too few slots (waiting lines build up)
    # 0.90 = 10% zu wenig Plätze (Stau explodiert)
SCENARIO_ADJUSTMENT = 0.95 

### Effective simulation capacity
    # number of slots to use
REAL_EFFICIENCY_FACTOR = BREAK_EVEN_FACTOR * SCENARIO_ADJUSTMENT
    # REAL_CAPACITY = number of parallel therapy places
REAL_CAPACITY = int(THEORETICAL_CAPACITY * REAL_EFFICIENCY_FACTOR)


# Demand growth (arrival rate)
    # At time 0, expected arrivals per week are TOTAL/52
INITIAL_WEEKLY_RATE = (TOTAL_PATIENTS_YEAR / WEEKS_PER_YEAR) * 1.0
GROWTH_PER_WEEK = 0.005  # Demand then grows continuously

stats_wait_times = []    # waiting times (in weeks) for patients who successfully get therapy

# -----------------------------
# Simulation processes
# -----------------------------
def patient_process(env, name, therapy_slots, dropped_counter):
    arrival_time = env.now  #patient arrives
    # With probability (1 - P_START_THERAPY) they do not start therapy and are counted as dropped
    if np.random.random() > PATIENT_START_THERAPY:
        dropped_counter['count'] += 1
        return 

    # request a therapy slot
    # If none are free, they wait until one is available
    with therapy_slots.request() as req:
        yield req 
        # Get a place -> Save waiting time
        stats_wait_times.append(env.now - arrival_time) # waiting time is env.now - arrival_time.
        # Patient occupies the slot for REAL_DURATION_WEEKS
        yield env.timeout(REAL_DURATION_WEEKS)

## At time 0, the system starts fully occupied, as if there are already ongoing therapies
def pre_fill_slot(env, therapy_slots, rng):
    with therapy_slots.request() as req:
        yield req
        # Each pre-filled slot is released after a random “remaining duration” 
        # between 0 and the full therapy length
        yield env.timeout(rng.uniform(0, REAL_DURATION_WEEKS))
# -> avoids artificially low waiting times at the beginning of the simulation

def setup(env, capacity, rng):
    # number of parallel slots
    therapy_slots = simpy.Resource(env, capacity=capacity)
    
    # Warm-up
    for _ in range(capacity):
        env.process(pre_fill_slot(env, therapy_slots, rng))
        
    dropped_counter = {'count': 0}
    i = 0
    current_rate = INITIAL_WEEKLY_RATE

    # Arrivals 
    # Interarrival times are exponential 
    # arrivals follow a Poisson process with rate current_rate
    while env.now < SIMULATION_DURATION:
        scale = 1.0 / current_rate
        interarrival_time = rng.exponential(scale=scale)
        yield env.timeout(interarrival_time)
        
        env.process(patient_process(env, f"Patient_{i}", therapy_slots, dropped_counter))
        i += 1
        # Exponential growth in demand
        current_rate = current_rate * (1 + (GROWTH_PER_WEEK * interarrival_time))

# -----------------------------
# MAIN
# -----------------------------
## Prints summary
if __name__ == "__main__":
    print(f"----- CAPACITY CALCULATION -----")
    print(f"Demand (Yes Patients):       {YES_PATIENTS_YEAR}")
    print(f"Actual therapy duration:     {REAL_DURATION_WEEKS:.2f} weeks")
    print(f"Required slots (100%):       {int(REQUIRED_SLOTS_SATURATION)}")
    print(f"Theoretical slots:           {THEORETICAL_CAPACITY}")
    print(f"-"*60)
    print(f"Break-even factor:           {BREAK_EVEN_FACTOR:.4f} (this equals 100%)")
    print(f"-"*60)
    print(f"Scenario setting:            {SCENARIO_ADJUSTMENT*100:.1f}% of demand (under-supply)")
    print(f"-> Applied efficiency factor:{REAL_EFFICIENCY_FACTOR:.4f}")
    print(f"-> Effective capacity:       {REAL_CAPACITY} slots")
    print(f"="*60)
    
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()
    env.process(setup(env, capacity=REAL_CAPACITY, rng=rng))
    env.run(until=SIMULATION_DURATION)

    waits = np.array(stats_wait_times)
    n_started = len(waits)

    print(f"\n----- RESULTS -----")
    if n_started > 0:
        mean_weeks = np.mean(waits)
        median_weeks = np.median(waits)
        p90_weeks = np.quantile(waits, 0.90)

        print(f"Patients starting therapy:   {n_started}")
        print(f"Average waiting time:        {mean_weeks*7:.1f} days")
        print(f"Median waiting time:         {median_weeks*7:.1f} days")
        print(f"P90 waiting time (risk):     {p90_weeks*7:.1f} days")
        print(f"Maximum waiting time:        {np.max(waits)*7:.1f} days")
    else:
        print("No data available.")

----- CAPACITY CALCULATION -----
Demand (Yes Patients):       84557
Actual therapy duration:     29.71 weeks
Required slots (100%):       48318
Theoretical slots:           102060
------------------------------------------------------------
Break-even factor:           0.4734 (this equals 100%)
------------------------------------------------------------
Scenario setting:            95.0% of demand (under-supply)
-> Applied efficiency factor:0.4498
-> Effective capacity:       45902 slots

----- RESULTS -----
Patients starting therapy:   241014
Average waiting time:        122.6 days
Median waiting time:         105.4 days
P90 waiting time (risk):     268.7 days
Maximum waiting time:        317.4 days


### scenario_adjustment = 1,00

In [ ]:
import simpy
import numpy as np

# -----------------------------
# Configuration 
# -----------------------------
WEEKS_PER_YEAR = 52
SIMULATION_DURATION = int(WEEKS_PER_YEAR * 3) #simulation runs for 3 years, measured in weeks
SEED = 42

#------------------------------
## Input
TOTAL_PATIENTS_YEAR = 95201   # people “show up” per year
YES_PATIENTS_YEAR = 84557     # only YES_PATIENTS_YEAR actually start therapy
PATIENT_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR #random arrival starts therapy

## Therapy duration (in weeks)
SESSIONS_REQUIRED = 24        # Therapy needs 24 sessions
WORKING_WEEKS = 42            # 42 working weeks/year (vacation/sick leave removed)
TIME_STRETCH = WEEKS_PER_YEAR / WORKING_WEEKS # “session week” is stretched by 52/42
REAL_DURATION_WEEKS = SESSIONS_REQUIRED * TIME_STRETCH 
# a therapy course occupies a slot for about 29.7 week

## Capacity calculation
# theoretical maximum number of parallel therapy slots based on staff groups
SLOTS_PART_TIME = 5490 * 9
SLOTS_FULL_TIME = 3510 * 15
THEORETICAL_CAPACITY = SLOTS_PART_TIME + SLOTS_FULL_TIME # ~102.060

### Slots needed for 100% coverage
    # Each year YES_PATIENTS_YEAR therapy starts
    # Patient start occupies a slot for REAL_DURATION_WEEKS
    # Divide by 52 to convert to “average concurrent slots needed”
REQUIRED_SLOTS_SATURATION = (YES_PATIENTS_YEAR * REAL_DURATION_WEEKS) / WEEKS_PER_YEAR

### Break- even factor
    # theoretical capacity you’d need so that capacity equals demand (100% coverage)
    # converts that to an absolute number of slots 
BREAK_EVEN_FACTOR = REQUIRED_SLOTS_SATURATION / THEORETICAL_CAPACITY

### Scenario adjustment
    # 1.00 = Perfect (no waiting lines)
    # 0.95 = 5% too few slots (waiting lines build up)
    # 0.90 = 10% zu wenig Plätze (Stau explodiert)
SCENARIO_ADJUSTMENT = 1.00 

### Effective simulation capacity
    # number of slots to use
REAL_EFFICIENCY_FACTOR = BREAK_EVEN_FACTOR * SCENARIO_ADJUSTMENT
    # REAL_CAPACITY = number of parallel therapy places
REAL_CAPACITY = int(THEORETICAL_CAPACITY * REAL_EFFICIENCY_FACTOR)


# Demand growth (arrival rate)
    # At time 0, expected arrivals per week are TOTAL/52
INITIAL_WEEKLY_RATE = (TOTAL_PATIENTS_YEAR / WEEKS_PER_YEAR) * 1.0
GROWTH_PER_WEEK = 0.005  # Demand then grows continuously

stats_wait_times = []    # waiting times (in weeks) for patients who successfully get therapy

# -----------------------------
# Simulation processes
# -----------------------------
def patient_process(env, name, therapy_slots, dropped_counter):
    arrival_time = env.now  #patient arrives
    # With probability (1 - P_START_THERAPY) they do not start therapy and are counted as dropped
    if rng.random() > PATIENT_START_THERAPY:
        dropped_counter['count'] += 1
        return 

    # request a therapy slot
    # If none are free, they wait until one is available
    with therapy_slots.request() as req:
        yield req 
        # Get a place -> Save waiting time
        stats_wait_times.append(env.now - arrival_time) # waiting time is env.now - arrival_time.
        # Patient occupies the slot for REAL_DURATION_WEEKS
        yield env.timeout(REAL_DURATION_WEEKS)

## At time 0, the system starts fully occupied, as if there are already ongoing therapies
def pre_fill_slot(env, therapy_slots, rng):
    with therapy_slots.request() as req:
        yield req
        # Each pre-filled slot is released after a random “remaining duration” 
        # between 0 and the full therapy length
        yield env.timeout(rng.uniform(0, REAL_DURATION_WEEKS))
# -> avoids artificially low waiting times at the beginning of the simulation

def setup(env, capacity, rng):
    # number of parallel slots
    therapy_slots = simpy.Resource(env, capacity=capacity)
    
    # Warm-up
    for _ in range(capacity):
        env.process(pre_fill_slot(env, therapy_slots, rng))
        
    dropped_counter = {'count': 0}
    i = 0
    current_rate = INITIAL_WEEKLY_RATE

    # Arrivals 
    # Interarrival times are exponential 
    # arrivals follow a Poisson process with rate current_rate
    while env.now < SIMULATION_DURATION:
        scale = 1.0 / current_rate
        interarrival_time = rng.exponential(scale=scale)
        yield env.timeout(interarrival_time)
        
        env.process(patient_process(env, f"Patient_{i}", therapy_slots, dropped_counter))
        i += 1
        # Exponential growth in demand
        current_rate = current_rate * (1 + (GROWTH_PER_WEEK * interarrival_time))

# -----------------------------
# MAIN
# -----------------------------
## Prints summary
if __name__ == "__main__":
    print(f"----- CAPACITY CALCULATION -----")
    print(f"Demand (Yes Patients):       {YES_PATIENTS_YEAR}")
    print(f"Actual therapy duration:     {REAL_DURATION_WEEKS:.2f} weeks")
    print(f"Required slots (100%):       {int(REQUIRED_SLOTS_SATURATION)}")
    print(f"Theoretical slots:           {THEORETICAL_CAPACITY}")
    print(f"-"*60)
    print(f"Break-even factor:           {BREAK_EVEN_FACTOR:.4f} (this equals 100%)")
    print(f"-"*60)
    print(f"Scenario setting:            {SCENARIO_ADJUSTMENT*100:.1f}% of demand (under-supply)")
    print(f"-> Applied efficiency factor:{REAL_EFFICIENCY_FACTOR:.4f}")
    print(f"-> Effective capacity:       {REAL_CAPACITY} slots")
    print(f"="*60)
    
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()
    env.process(setup(env, capacity=REAL_CAPACITY, rng=rng))
    env.run(until=SIMULATION_DURATION)

    waits = np.array(stats_wait_times)
    n_started = len(waits)

    print(f"\n----- RESULTS -----")
    if n_started > 0:
        mean_weeks = np.mean(waits)
        median_weeks = np.median(waits)
        p90_weeks = np.quantile(waits, 0.90)

        print(f"Patients starting therapy:   {n_started}")
        print(f"Average waiting time:        {mean_weeks*7:.1f} days")
        print(f"Median waiting time:         {median_weeks*7:.1f} days")
        print(f"P90 waiting time (risk):     {p90_weeks*7:.1f} days")
        print(f"Maximum waiting time:        {np.max(waits)*7:.1f} days")
    else:
        print("No data available.")

----- CAPACITY CALCULATION -----
Demand (Yes Patients):       84557
Actual therapy duration:     29.71 weeks
Required slots (100%):       48318
Theoretical slots:           102060
------------------------------------------------------------
Break-even factor:           0.4734 (this equals 100%)
------------------------------------------------------------
Scenario setting:            100.0% of demand (under-supply)
-> Applied efficiency factor:0.4734
-> Effective capacity:       48318 slots

----- RESULTS -----
Patients starting therapy:   253692
Average waiting time:        105.4 days
Median waiting time:         87.8 days
P90 waiting time (risk):     240.8 days
Maximum waiting time:        286.3 days


### scenario_adjustment = 0,98

In [ ]:
import simpy
import numpy as np

# -----------------------------
# Configuration 
# -----------------------------
WEEKS_PER_YEAR = 52
SIMULATION_DURATION = int(WEEKS_PER_YEAR * 3) #simulation runs for 3 years, measured in weeks
SEED = 42

#------------------------------
## Input
TOTAL_PATIENTS_YEAR = 95201   # people “show up” per year
YES_PATIENTS_YEAR = 84557     # only YES_PATIENTS_YEAR actually start therapy
PATIENT_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR #random arrival starts therapy

## Therapy duration (in weeks)
SESSIONS_REQUIRED = 24        # Therapy needs 24 sessions
WORKING_WEEKS = 42            # 42 working weeks/year (vacation/sick leave removed)
TIME_STRETCH = WEEKS_PER_YEAR / WORKING_WEEKS # “session week” is stretched by 52/42
REAL_DURATION_WEEKS = SESSIONS_REQUIRED * TIME_STRETCH 
# a therapy course occupies a slot for about 29.7 week

## Capacity calculation
# theoretical maximum number of parallel therapy slots based on staff groups
SLOTS_PART_TIME = 5490 * 9
SLOTS_FULL_TIME = 3510 * 15
THEORETICAL_CAPACITY = SLOTS_PART_TIME + SLOTS_FULL_TIME # ~102.060

### Slots needed for 100% coverage
    # Each year YES_PATIENTS_YEAR therapy starts
    # Patient start occupies a slot for REAL_DURATION_WEEKS
    # Divide by 52 to convert to “average concurrent slots needed”
REQUIRED_SLOTS_SATURATION = (YES_PATIENTS_YEAR * REAL_DURATION_WEEKS) / WEEKS_PER_YEAR

### Break- even factor
    # theoretical capacity you’d need so that capacity equals demand (100% coverage)
    # converts that to an absolute number of slots 
BREAK_EVEN_FACTOR = REQUIRED_SLOTS_SATURATION / THEORETICAL_CAPACITY

### Scenario adjustment
    # 1.00 = Perfect (no waiting lines)
    # 0.95 = 5% too few slots (waiting lines build up)
    # 0.90 = 10% zu wenig Plätze (Stau explodiert)
SCENARIO_ADJUSTMENT = 0.98 

### Effective simulation capacity
    # number of slots to use
REAL_EFFICIENCY_FACTOR = BREAK_EVEN_FACTOR * SCENARIO_ADJUSTMENT
    # REAL_CAPACITY = number of parallel therapy places
REAL_CAPACITY = int(THEORETICAL_CAPACITY * REAL_EFFICIENCY_FACTOR)


# Demand growth (arrival rate)
    # At time 0, expected arrivals per week are TOTAL/52
INITIAL_WEEKLY_RATE = (TOTAL_PATIENTS_YEAR / WEEKS_PER_YEAR) * 1.0
GROWTH_PER_WEEK = 0.005  # Demand then grows continuously

stats_wait_times = []    # waiting times (in weeks) for patients who successfully get therapy

# -----------------------------
# Simulation processes
# -----------------------------
def patient_process(env, name, therapy_slots, dropped_counter):
    arrival_time = env.now  #patient arrives
    # With probability (1 - P_START_THERAPY) they do not start therapy and are counted as dropped
    if rng.random() > PATIENT_START_THERAPY:
        dropped_counter['count'] += 1
        return 

    # request a therapy slot
    # If none are free, they wait until one is available
    with therapy_slots.request() as req:
        yield req 
        # Get a place -> Save waiting time
        stats_wait_times.append(env.now - arrival_time) # waiting time is env.now - arrival_time.
        # Patient occupies the slot for REAL_DURATION_WEEKS
        yield env.timeout(REAL_DURATION_WEEKS)

## At time 0, the system starts fully occupied, as if there are already ongoing therapies
def pre_fill_slot(env, therapy_slots, rng):
    with therapy_slots.request() as req:
        yield req
        # Each pre-filled slot is released after a random “remaining duration” 
        # between 0 and the full therapy length
        yield env.timeout(rng.uniform(0, REAL_DURATION_WEEKS))
# -> avoids artificially low waiting times at the beginning of the simulation

def setup(env, capacity, rng):
    # number of parallel slots
    therapy_slots = simpy.Resource(env, capacity=capacity)
    
    # Warm-up
    for _ in range(capacity):
        env.process(pre_fill_slot(env, therapy_slots, rng))
        
    dropped_counter = {'count': 0}
    i = 0
    current_rate = INITIAL_WEEKLY_RATE

    # Arrivals 
    # Interarrival times are exponential 
    # arrivals follow a Poisson process with rate current_rate
    while env.now < SIMULATION_DURATION:
        scale = 1.0 / current_rate
        interarrival_time = rng.exponential(scale=scale)
        yield env.timeout(interarrival_time)
        
        env.process(patient_process(env, f"Patient_{i}", therapy_slots, dropped_counter))
        i += 1
        # Exponential growth in demand
        current_rate = current_rate * (1 + (GROWTH_PER_WEEK * interarrival_time))

# -----------------------------
# MAIN
# -----------------------------
## Prints summary
if __name__ == "__main__":
    print(f"----- CAPACITY CALCULATION -----")
    print(f"Demand (Yes Patients):       {YES_PATIENTS_YEAR}")
    print(f"Actual therapy duration:     {REAL_DURATION_WEEKS:.2f} weeks")
    print(f"Required slots (100%):       {int(REQUIRED_SLOTS_SATURATION)}")
    print(f"Theoretical slots:           {THEORETICAL_CAPACITY}")
    print(f"-"*60)
    print(f"Break-even factor:           {BREAK_EVEN_FACTOR:.4f} (this equals 100%)")
    print(f"-"*60)
    print(f"Scenario setting:            {SCENARIO_ADJUSTMENT*100:.1f}% of demand (under-supply)")
    print(f"-> Applied efficiency factor:{REAL_EFFICIENCY_FACTOR:.4f}")
    print(f"-> Effective capacity:       {REAL_CAPACITY} slots")
    print(f"="*60)
    
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()
    env.process(setup(env, capacity=REAL_CAPACITY, rng=rng))
    env.run(until=SIMULATION_DURATION)

    waits = np.array(stats_wait_times)
    n_started = len(waits)

    print(f"\n----- RESULTS -----")
    if n_started > 0:
        mean_weeks = np.mean(waits)
        median_weeks = np.median(waits)
        p90_weeks = np.quantile(waits, 0.90)

        print(f"Patients starting therapy:   {n_started}")
        print(f"Average waiting time:        {mean_weeks*7:.1f} days")
        print(f"Median waiting time:         {median_weeks*7:.1f} days")
        print(f"P90 waiting time (risk):     {p90_weeks*7:.1f} days")
        print(f"Maximum waiting time:        {np.max(waits)*7:.1f} days")
    else:
        print("No data available.")

----- CAPACITY CALCULATION -----
Demand (Yes Patients):       84557
Actual therapy duration:     29.71 weeks
Required slots (100%):       48318
Theoretical slots:           102060
------------------------------------------------------------
Break-even factor:           0.4734 (this equals 100%)
------------------------------------------------------------
Scenario setting:            98.0% of demand (under-supply)
-> Applied efficiency factor:0.4640
-> Effective capacity:       47351 slots

----- RESULTS -----
Patients starting therapy:   248600
Average waiting time:        112.8 days
Median waiting time:         95.8 days
P90 waiting time (risk):     252.6 days
Maximum waiting time:        298.9 days


### scenario_adjustment = 0,95

In [ ]:
import simpy
import numpy as np

# -----------------------------
# Configuration 
# -----------------------------
WEEKS_PER_YEAR = 52
SIMULATION_DURATION = int(WEEKS_PER_YEAR * 3) #simulation runs for 3 years, measured in weeks
SEED = 42

#------------------------------
## Input
TOTAL_PATIENTS_YEAR = 95201   # people “show up” per year
YES_PATIENTS_YEAR = 84557     # only YES_PATIENTS_YEAR actually start therapy
PATIENT_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR #random arrival starts therapy

## Therapy duration (in weeks)
SESSIONS_REQUIRED = 24        # Therapy needs 24 sessions
WORKING_WEEKS = 42            # 42 working weeks/year (vacation/sick leave removed)
TIME_STRETCH = WEEKS_PER_YEAR / WORKING_WEEKS # “session week” is stretched by 52/42
REAL_DURATION_WEEKS = SESSIONS_REQUIRED * TIME_STRETCH 
# a therapy course occupies a slot for about 29.7 week

## Capacity calculation
# theoretical maximum number of parallel therapy slots based on staff groups
SLOTS_PART_TIME = 5490 * 9
SLOTS_FULL_TIME = 3510 * 15
THEORETICAL_CAPACITY = SLOTS_PART_TIME + SLOTS_FULL_TIME # ~102.060

### Slots needed for 100% coverage
    # Each year YES_PATIENTS_YEAR therapy starts
    # Patient start occupies a slot for REAL_DURATION_WEEKS
    # Divide by 52 to convert to “average concurrent slots needed”
REQUIRED_SLOTS_SATURATION = (YES_PATIENTS_YEAR * REAL_DURATION_WEEKS) / WEEKS_PER_YEAR

### Break- even factor
    # theoretical capacity you’d need so that capacity equals demand (100% coverage)
    # converts that to an absolute number of slots 
BREAK_EVEN_FACTOR = REQUIRED_SLOTS_SATURATION / THEORETICAL_CAPACITY

### Scenario adjustment
    # 1.00 = Perfect (no waiting lines)
    # 0.95 = 5% too few slots (waiting lines build up)
    # 0.90 = 10% zu wenig Plätze (Stau explodiert)
SCENARIO_ADJUSTMENT = 0.95 

### Effective simulation capacity
    # number of slots to use
REAL_EFFICIENCY_FACTOR = BREAK_EVEN_FACTOR * SCENARIO_ADJUSTMENT
    # REAL_CAPACITY = number of parallel therapy places
REAL_CAPACITY = int(THEORETICAL_CAPACITY * REAL_EFFICIENCY_FACTOR)


# Demand growth (arrival rate)
    # At time 0, expected arrivals per week are TOTAL/52
INITIAL_WEEKLY_RATE = (TOTAL_PATIENTS_YEAR / WEEKS_PER_YEAR) * 1.0
GROWTH_PER_WEEK = 0.005  # Demand then grows continuously

stats_wait_times = []    # waiting times (in weeks) for patients who successfully get therapy

# -----------------------------
# Simulation processes
# -----------------------------
def patient_process(env, name, therapy_slots, dropped_counter):
    arrival_time = env.now  #patient arrives
    # With probability (1 - P_START_THERAPY) they do not start therapy and are counted as dropped
    if rng.random() > PATIENT_START_THERAPY:
        dropped_counter['count'] += 1
        return 

    # request a therapy slot
    # If none are free, they wait until one is available
    with therapy_slots.request() as req:
        yield req 
        # Get a place -> Save waiting time
        stats_wait_times.append(env.now - arrival_time) # waiting time is env.now - arrival_time.
        # Patient occupies the slot for REAL_DURATION_WEEKS
        yield env.timeout(REAL_DURATION_WEEKS)

## At time 0, the system starts fully occupied, as if there are already ongoing therapies
def pre_fill_slot(env, therapy_slots, rng):
    with therapy_slots.request() as req:
        yield req
        # Each pre-filled slot is released after a random “remaining duration” 
        # between 0 and the full therapy length
        yield env.timeout(rng.uniform(0, REAL_DURATION_WEEKS))
# -> avoids artificially low waiting times at the beginning of the simulation

def setup(env, capacity, rng):
    # number of parallel slots
    therapy_slots = simpy.Resource(env, capacity=capacity)
    
    # Warm-up
    for _ in range(capacity):
        env.process(pre_fill_slot(env, therapy_slots, rng))
        
    dropped_counter = {'count': 0}
    i = 0
    current_rate = INITIAL_WEEKLY_RATE

    # Arrivals 
    # Interarrival times are exponential 
    # arrivals follow a Poisson process with rate current_rate
    while env.now < SIMULATION_DURATION:
        scale = 1.0 / current_rate
        interarrival_time = rng.exponential(scale=scale)
        yield env.timeout(interarrival_time)
        
        env.process(patient_process(env, f"Patient_{i}", therapy_slots, dropped_counter))
        i += 1
        # Exponential growth in demand
        current_rate = current_rate * (1 + (GROWTH_PER_WEEK * interarrival_time))

# -----------------------------
# MAIN
# -----------------------------
## Prints summary
if __name__ == "__main__":
    print(f"----- CAPACITY CALCULATION -----")
    print(f"Demand (Yes Patients):       {YES_PATIENTS_YEAR}")
    print(f"Actual therapy duration:     {REAL_DURATION_WEEKS:.2f} weeks")
    print(f"Required slots (100%):       {int(REQUIRED_SLOTS_SATURATION)}")
    print(f"Theoretical slots:           {THEORETICAL_CAPACITY}")
    print(f"-"*60)
    print(f"Break-even factor:           {BREAK_EVEN_FACTOR:.4f} (this equals 100%)")
    print(f"-"*60)
    print(f"Scenario setting:            {SCENARIO_ADJUSTMENT*100:.1f}% of demand (under-supply)")
    print(f"-> Applied efficiency factor:{REAL_EFFICIENCY_FACTOR:.4f}")
    print(f"-> Effective capacity:       {REAL_CAPACITY} slots")
    print(f"="*60)
    
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()
    env.process(setup(env, capacity=REAL_CAPACITY, rng=rng))
    env.run(until=SIMULATION_DURATION)

    waits = np.array(stats_wait_times)
    n_started = len(waits)

    print(f"\n----- RESULTS -----")
    if n_started > 0:
        mean_weeks = np.mean(waits)
        median_weeks = np.median(waits)
        p90_weeks = np.quantile(waits, 0.90)

        print(f"Patients starting therapy:   {n_started}")
        print(f"Average waiting time:        {mean_weeks*7:.1f} days")
        print(f"Median waiting time:         {median_weeks*7:.1f} days")
        print(f"P90 waiting time (risk):     {p90_weeks*7:.1f} days")
        print(f"Maximum waiting time:        {np.max(waits)*7:.1f} days")
    else:
        print("No data available.")

----- CAPACITY CALCULATION -----
Demand (Yes Patients):       84557
Actual therapy duration:     29.71 weeks
Required slots (100%):       48318
Theoretical slots:           102060
------------------------------------------------------------
Break-even factor:           0.4734 (this equals 100%)
------------------------------------------------------------
Scenario setting:            95.0% of demand (under-supply)
-> Applied efficiency factor:0.4498
-> Effective capacity:       45902 slots

----- RESULTS -----
Patients starting therapy:   241014
Average waiting time:        123.8 days
Median waiting time:         107.6 days
P90 waiting time (risk):     270.3 days
Maximum waiting time:        317.8 days


### scenario_adjustment = 0,90

In [ ]:
import simpy
import numpy as np

# -----------------------------
# Configuration 
# -----------------------------
WEEKS_PER_YEAR = 52
SIMULATION_DURATION = int(WEEKS_PER_YEAR * 3) #simulation runs for 3 years, measured in weeks
SEED = 42

#------------------------------
## Input
TOTAL_PATIENTS_YEAR = 95201   # people “show up” per year
YES_PATIENTS_YEAR = 84557     # only YES_PATIENTS_YEAR actually start therapy
PATIENT_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR #random arrival starts therapy

## Therapy duration (in weeks)
SESSIONS_REQUIRED = 24        # Therapy needs 24 sessions
WORKING_WEEKS = 42            # 42 working weeks/year (vacation/sick leave removed)
TIME_STRETCH = WEEKS_PER_YEAR / WORKING_WEEKS # “session week” is stretched by 52/42
REAL_DURATION_WEEKS = SESSIONS_REQUIRED * TIME_STRETCH 
# a therapy course occupies a slot for about 29.7 week

## Capacity calculation
# theoretical maximum number of parallel therapy slots based on staff groups
SLOTS_PART_TIME = 5490 * 9
SLOTS_FULL_TIME = 3510 * 15
THEORETICAL_CAPACITY = SLOTS_PART_TIME + SLOTS_FULL_TIME # ~102.060

### Slots needed for 100% coverage
    # Each year YES_PATIENTS_YEAR therapy starts
    # Patient start occupies a slot for REAL_DURATION_WEEKS
    # Divide by 52 to convert to “average concurrent slots needed”
REQUIRED_SLOTS_SATURATION = (YES_PATIENTS_YEAR * REAL_DURATION_WEEKS) / WEEKS_PER_YEAR

### Break- even factor
    # theoretical capacity you’d need so that capacity equals demand (100% coverage)
    # converts that to an absolute number of slots 
BREAK_EVEN_FACTOR = REQUIRED_SLOTS_SATURATION / THEORETICAL_CAPACITY

### Scenario adjustment
    # 1.00 = Perfect (no waiting lines)
    # 0.95 = 5% too few slots (waiting lines build up)
    # 0.90 = 10% zu wenig Plätze (Stau explodiert)
SCENARIO_ADJUSTMENT = 0.90 

### Effective simulation capacity
    # number of slots to use
REAL_EFFICIENCY_FACTOR = BREAK_EVEN_FACTOR * SCENARIO_ADJUSTMENT
    # REAL_CAPACITY = number of parallel therapy places
REAL_CAPACITY = int(THEORETICAL_CAPACITY * REAL_EFFICIENCY_FACTOR)


# Demand growth (arrival rate)
    # At time 0, expected arrivals per week are TOTAL/52
INITIAL_WEEKLY_RATE = (TOTAL_PATIENTS_YEAR / WEEKS_PER_YEAR) * 1.0
GROWTH_PER_WEEK = 0.005  # Demand then grows continuously

stats_wait_times = []    # waiting times (in weeks) for patients who successfully get therapy

# -----------------------------
# Simulation processes
# -----------------------------
def patient_process(env, name, therapy_slots, dropped_counter):
    arrival_time = env.now  #patient arrives
    # With probability (1 - P_START_THERAPY) they do not start therapy and are counted as dropped
    if rng.random() > PATIENT_START_THERAPY:
        dropped_counter['count'] += 1
        return 

    # request a therapy slot
    # If none are free, they wait until one is available
    with therapy_slots.request() as req:
        yield req 
        # Get a place -> Save waiting time
        stats_wait_times.append(env.now - arrival_time) # waiting time is env.now - arrival_time.
        # Patient occupies the slot for REAL_DURATION_WEEKS
        yield env.timeout(REAL_DURATION_WEEKS)

## At time 0, the system starts fully occupied, as if there are already ongoing therapies
def pre_fill_slot(env, therapy_slots, rng):
    with therapy_slots.request() as req:
        yield req
        # Each pre-filled slot is released after a random “remaining duration” 
        # between 0 and the full therapy length
        yield env.timeout(rng.uniform(0, REAL_DURATION_WEEKS))
# -> avoids artificially low waiting times at the beginning of the simulation

def setup(env, capacity, rng):
    # number of parallel slots
    therapy_slots = simpy.Resource(env, capacity=capacity)
    
    # Warm-up
    for _ in range(capacity):
        env.process(pre_fill_slot(env, therapy_slots, rng))
        
    dropped_counter = {'count': 0}
    i = 0
    current_rate = INITIAL_WEEKLY_RATE

    # Arrivals 
    # Interarrival times are exponential 
    # arrivals follow a Poisson process with rate current_rate
    while env.now < SIMULATION_DURATION:
        scale = 1.0 / current_rate
        interarrival_time = rng.exponential(scale=scale)
        yield env.timeout(interarrival_time)
        
        env.process(patient_process(env, f"Patient_{i}", therapy_slots, dropped_counter))
        i += 1
        # Exponential growth in demand
        current_rate = current_rate * (1 + (GROWTH_PER_WEEK * interarrival_time))

# -----------------------------
# MAIN
# -----------------------------
## Prints summary
if __name__ == "__main__":
    print(f"----- CAPACITY CALCULATION -----")
    print(f"Demand (Yes Patients):       {YES_PATIENTS_YEAR}")
    print(f"Actual therapy duration:     {REAL_DURATION_WEEKS:.2f} weeks")
    print(f"Required slots (100%):       {int(REQUIRED_SLOTS_SATURATION)}")
    print(f"Theoretical slots:           {THEORETICAL_CAPACITY}")
    print(f"-"*60)
    print(f"Break-even factor:           {BREAK_EVEN_FACTOR:.4f} (this equals 100%)")
    print(f"-"*60)
    print(f"Scenario setting:            {SCENARIO_ADJUSTMENT*100:.1f}% of demand (under-supply)")
    print(f"-> Applied efficiency factor:{REAL_EFFICIENCY_FACTOR:.4f}")
    print(f"-> Effective capacity:       {REAL_CAPACITY} slots")
    print(f"="*60)
    
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()
    env.process(setup(env, capacity=REAL_CAPACITY, rng=rng))
    env.run(until=SIMULATION_DURATION)

    waits = np.array(stats_wait_times)
    n_started = len(waits)

    print(f"\n----- RESULTS -----")
    if n_started > 0:
        mean_weeks = np.mean(waits)
        median_weeks = np.median(waits)
        p90_weeks = np.quantile(waits, 0.90)

        print(f"Patients starting therapy:   {n_started}")
        print(f"Average waiting time:        {mean_weeks*7:.1f} days")
        print(f"Median waiting time:         {median_weeks*7:.1f} days")
        print(f"P90 waiting time (risk):     {p90_weeks*7:.1f} days")
        print(f"Maximum waiting time:        {np.max(waits)*7:.1f} days")
    else:
        print("No data available.")

----- CAPACITY CALCULATION -----
Demand (Yes Patients):       84557
Actual therapy duration:     29.71 weeks
Required slots (100%):       48318
Theoretical slots:           102060
------------------------------------------------------------
Break-even factor:           0.4734 (this equals 100%)
------------------------------------------------------------
Scenario setting:            90.0% of demand (under-supply)
-> Applied efficiency factor:0.4261
-> Effective capacity:       43486 slots

----- RESULTS -----
Patients starting therapy:   228337
Average waiting time:        142.6 days
Median waiting time:         127.4 days
P90 waiting time (risk):     300.3 days
Maximum waiting time:        350.0 days


### scenario_adjustment = 0,85

In [ ]:
import simpy
import numpy as np

# -----------------------------
# Configuration 
# -----------------------------
WEEKS_PER_YEAR = 52
SIMULATION_DURATION = int(WEEKS_PER_YEAR * 3) #simulation runs for 3 years, measured in weeks
SEED = 42

#------------------------------
## Input
TOTAL_PATIENTS_YEAR = 95201   # people “show up” per year
YES_PATIENTS_YEAR = 84557     # only YES_PATIENTS_YEAR actually start therapy
PATIENT_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR #random arrival starts therapy

## Therapy duration (in weeks)
SESSIONS_REQUIRED = 24        # Therapy needs 24 sessions
WORKING_WEEKS = 42            # 42 working weeks/year (vacation/sick leave removed)
TIME_STRETCH = WEEKS_PER_YEAR / WORKING_WEEKS # “session week” is stretched by 52/42
REAL_DURATION_WEEKS = SESSIONS_REQUIRED * TIME_STRETCH 
# a therapy course occupies a slot for about 29.7 week

## Capacity calculation
# theoretical maximum number of parallel therapy slots based on staff groups
SLOTS_PART_TIME = 5490 * 9
SLOTS_FULL_TIME = 3510 * 15
THEORETICAL_CAPACITY = SLOTS_PART_TIME + SLOTS_FULL_TIME # ~102.060

### Slots needed for 100% coverage
    # Each year YES_PATIENTS_YEAR therapy starts
    # Patient start occupies a slot for REAL_DURATION_WEEKS
    # Divide by 52 to convert to “average concurrent slots needed”
REQUIRED_SLOTS_SATURATION = (YES_PATIENTS_YEAR * REAL_DURATION_WEEKS) / WEEKS_PER_YEAR

### Break- even factor
    # theoretical capacity you’d need so that capacity equals demand (100% coverage)
    # converts that to an absolute number of slots 
BREAK_EVEN_FACTOR = REQUIRED_SLOTS_SATURATION / THEORETICAL_CAPACITY

### Scenario adjustment
    # 1.00 = Perfect (no waiting lines)
    # 0.95 = 5% too few slots (waiting lines build up)
    # 0.90 = 10% zu wenig Plätze (Stau explodiert)
SCENARIO_ADJUSTMENT = 0.85 

### Effective simulation capacity
    # number of slots to use
REAL_EFFICIENCY_FACTOR = BREAK_EVEN_FACTOR * SCENARIO_ADJUSTMENT
    # REAL_CAPACITY = number of parallel therapy places
REAL_CAPACITY = int(THEORETICAL_CAPACITY * REAL_EFFICIENCY_FACTOR)


# Demand growth (arrival rate)
    # At time 0, expected arrivals per week are TOTAL/52
INITIAL_WEEKLY_RATE = (TOTAL_PATIENTS_YEAR / WEEKS_PER_YEAR) * 1.0
GROWTH_PER_WEEK = 0.005  # Demand then grows continuously

stats_wait_times = []    # waiting times (in weeks) for patients who successfully get therapy

# -----------------------------
# Simulation processes
# -----------------------------
def patient_process(env, name, therapy_slots, dropped_counter):
    arrival_time = env.now  #patient arrives
    # With probability (1 - P_START_THERAPY) they do not start therapy and are counted as dropped
    if rng.random() > PATIENT_START_THERAPY:
        dropped_counter['count'] += 1
        return 

    # request a therapy slot
    # If none are free, they wait until one is available
    with therapy_slots.request() as req:
        yield req 
        # Get a place -> Save waiting time
        stats_wait_times.append(env.now - arrival_time) # waiting time is env.now - arrival_time.
        # Patient occupies the slot for REAL_DURATION_WEEKS
        yield env.timeout(REAL_DURATION_WEEKS)

## At time 0, the system starts fully occupied, as if there are already ongoing therapies
def pre_fill_slot(env, therapy_slots, rng):
    with therapy_slots.request() as req:
        yield req
        # Each pre-filled slot is released after a random “remaining duration” 
        # between 0 and the full therapy length
        yield env.timeout(rng.uniform(0, REAL_DURATION_WEEKS))
# -> avoids artificially low waiting times at the beginning of the simulation

def setup(env, capacity, rng):
    # number of parallel slots
    therapy_slots = simpy.Resource(env, capacity=capacity)
    
    # Warm-up
    for _ in range(capacity):
        env.process(pre_fill_slot(env, therapy_slots, rng))
        
    dropped_counter = {'count': 0}
    i = 0
    current_rate = INITIAL_WEEKLY_RATE

    # Arrivals 
    # Interarrival times are exponential 
    # arrivals follow a Poisson process with rate current_rate
    while env.now < SIMULATION_DURATION:
        scale = 1.0 / current_rate
        interarrival_time = rng.exponential(scale=scale)
        yield env.timeout(interarrival_time)
        
        env.process(patient_process(env, f"Patient_{i}", therapy_slots, dropped_counter))
        i += 1
        # Exponential growth in demand
        current_rate = current_rate * (1 + (GROWTH_PER_WEEK * interarrival_time))

# -----------------------------
# MAIN
# -----------------------------
## Prints summary
if __name__ == "__main__":
    print(f"----- CAPACITY CALCULATION -----")
    print(f"Demand (Yes Patients):       {YES_PATIENTS_YEAR}")
    print(f"Actual therapy duration:     {REAL_DURATION_WEEKS:.2f} weeks")
    print(f"Required slots (100%):       {int(REQUIRED_SLOTS_SATURATION)}")
    print(f"Theoretical slots:           {THEORETICAL_CAPACITY}")
    print(f"-"*60)
    print(f"Break-even factor:           {BREAK_EVEN_FACTOR:.4f} (this equals 100%)")
    print(f"-"*60)
    print(f"Scenario setting:            {SCENARIO_ADJUSTMENT*100:.1f}% of demand (under-supply)")
    print(f"-> Applied efficiency factor:{REAL_EFFICIENCY_FACTOR:.4f}")
    print(f"-> Effective capacity:       {REAL_CAPACITY} slots")
    print(f"="*60)
    
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()
    env.process(setup(env, capacity=REAL_CAPACITY, rng=rng))
    env.run(until=SIMULATION_DURATION)

    waits = np.array(stats_wait_times)
    n_started = len(waits)

    print(f"\n----- RESULTS -----")
    if n_started > 0:
        mean_weeks = np.mean(waits)
        median_weeks = np.median(waits)
        p90_weeks = np.quantile(waits, 0.90)

        print(f"Patients starting therapy:   {n_started}")
        print(f"Average waiting time:        {mean_weeks*7:.1f} days")
        print(f"Median waiting time:         {median_weeks*7:.1f} days")
        print(f"P90 waiting time (risk):     {p90_weeks*7:.1f} days")
        print(f"Maximum waiting time:        {np.max(waits)*7:.1f} days")
    else:
        print("No data available.")

----- CAPACITY CALCULATION -----
Demand (Yes Patients):       84557
Actual therapy duration:     29.71 weeks
Required slots (100%):       48318
Theoretical slots:           102060
------------------------------------------------------------
Break-even factor:           0.4734 (this equals 100%)
------------------------------------------------------------
Scenario setting:            85.0% of demand (under-supply)
-> Applied efficiency factor:0.4024
-> Effective capacity:       41070 slots

----- RESULTS -----
Patients starting therapy:   215647
Average waiting time:        161.4 days
Median waiting time:         147.3 days
P90 waiting time (risk):     330.6 days
Maximum waiting time:        382.5 days
